# Operating Systems & Concurrency: writing multi-threaded programs
_COSC 208, Introduction to Computer Systems, 2024-04-22_

## Announcements
* Project 3 due Wednesday @ 11pm

## Outline
* Warm-up
* Pthreads API
* Creating multiple threads
* Returning values from threads
* Practice writing multi-threaded programs

## Warm-up

A program contains the following functions:
```C
void *dbl(void *arg) {
    int *t = (int *)arg;
    *t = *t * 2;
}

void *inc(void *arg) {
    int *t = (int *)arg;
    *t = *t + 1;
}
```
_For each of the following main methods, list **all possible outputs** the program could produce. Assume threads are only preempted if they become blocked waiting for other threads._

In [ ]:
// Q1
int main() {
    int *total = malloc(sizeof(int));
    *total = 3;
    pthread_t thrA, thrB;
    pthread_create(&thrA, NULL, &dbl, total);
    pthread_create(&thrB, NULL, &inc, total);
    pthread_join(thrA, NULL);
    pthread_join(thrB, NULL);
    printf("%d\n", total);
}

* `7` (if `thrB` runs after `thrA` finishes)
* `8` (if `thrA` runs after `thrB` finishes)

<p style="height:1em;"></p>

In [ ]:
// Q2
int main() {
    int *total = malloc(sizeof(int));
    *total = 3
    pthread_t thrA, thrB;
    pthread_create(&thrA, NULL, &inc, total);
    pthread_join(thrA, NULL);
    pthread_create(&thrB, NULL, &dbl, total);
    pthread_join(thrB, NULL);
    printf("%d\n", total);
}

* `8` (`thrA` is joined, i.e., must finish, before `thrB` is created) 

<div style="height:1em;"></div>

🛑 **STOP here** after completing the above questions.

<div style="page-break-after:always;"></div>

## Creating multiple threads

* Create an array of `pthread_t` and an array of arguments
* Call `pthread_create` within a loop
* Call `pthread_join` within a separate loop
* Example

In [36]:
/* 1*/  #include <pthread.h>
/* 2*/  #include <stdio.h>
/* 3*/  #include <stdlib.h>
/* 4*/  #define NUM_THREADS 5
/* 5*/  void *simple(void *arg) {
/* 6*/      int *id = (int *)arg;
/* 7*/      printf("I am thread %d\n", *id);
/* 8*/      return NULL;
/* 9*/  }
/*10*/  int main() {
/*11*/      pthread_t threads[NUM_THREADS];
/*12*/      int ids[NUM_THREADS];
/*13*/      for (int i = 0; i < NUM_THREADS; i++) {
/*14*/          ids[i] = i+1;
/*15*/          pthread_create(threads+i, NULL, &simple, ids+i);
/*16*/      } 
/*17*/      for (int i = 0; i < NUM_THREADS; i++) {
/*18*/          pthread_join(threads[i], NULL);
/*19*/      }
/*20*/      printf("All threads finished\n");
/*21*/  }

I am thread 1
I am thread 2
I am thread 3
I am thread 4
I am thread 5
All threads finished


## Returning values from threads

* _When does a thread end?_ — when the function passed to `pthread_create` finishes (i.e., returns)
* _What happens to a function's parameters and local variables when the function returns?_ — they no longer exist (i.e., the stack frame is destroyed)
* _Where should we store a value that should exist even after a function returns?_ — on the heap
* Need to store a thread's return value on the heap
* Thread returns a pointer to the value on the heap
* Example

In [20]:
/* 1*/  #include <stdio.h>
/* 2*/  #include <stdlib.h>
/* 3*/  #include <string.h>
/* 4*/  #include <pthread.h>
/* 5*/  void *length(void *arg) {
/* 6*/      char *str = (char *)arg;
/* 7*/      int *len = malloc(sizeof(int));
/* 8*/      *len = strlen(str);
/* 9*/      return len;
/*10*/  }
/*11*/  int main() {
/*12*/      pthread_t thread;
/*13*/      char *phrase = "Hello, threads!";
/*14*/      pthread_create(&thread, NULL, &length, phrase);
/*15*/      int *result = NULL;
/*16*/      pthread_join(thread, (void **)&result);
/*17*/      printf("Length: %d\n", *result);
/*18*/      free(result);
/*19*/  }

Length: 15


* `pthread_join` returns `0` if successful, or an error number
* To get the pointer returned by the thread, we need to pass a location where the pointer can be stored — i.e., we need to pass a double pointer

<div style="page-break-after:always;"></div>

## Practice writing multi-threaded programs

* Q3: _Write a function called `sum_array` which takes an array of `ARRAY_LEN` integers and returns the sum of the integers. Your function should have the appropriate prototype/implementation to serve as the entry point for a thread. Assume `ARRAY_LEN` is a constant which has been `#define`d._

<p style="height:20em;"></p>

* Q4: _Write a function called `sum_matrix` which takes an array of `NUM_ARRAYS` arrays of integers (i.e., an `int **`) and returns the sum of all the integers. The function should create `NUM_ARRAYS` threads, each running the `sum_array` function for a single array of integers. Assume `NUM_ARRAYS` is a constant which has been `#define`d._

In [2]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

#define ARRAY_LEN 10
#define NUM_ARRAYS 5

void *sum_array(void *args) {
    int *nums = (int *)args;
    int *sum = malloc(sizeof(int));
    *sum = 0;
    for (int i = 0; i < ARRAY_LEN; i++) {
        *sum += nums[i];
    }
    return sum;
}

int sum_matrix(int *matrix[]) {
    pthread_t threads[NUM_ARRAYS];
    for (int i = 0; i < NUM_ARRAYS; i++) {
        pthread_create(&(threads[i]), NULL, &sum_array, matrix[i]);
    }

    int total = 0;
    for (int i = 0; i < NUM_ARRAYS; i++) {
        int *sum;
        pthread_join(threads[i], (void **)(&sum));
        total += *sum;
        free(sum);
    }

    return total;
}

int main() {
    int *matrix[NUM_ARRAYS];
    for (int i = 0; i < NUM_ARRAYS; i++) {
        matrix[i] = malloc(sizeof(int) * ARRAY_LEN);
        for (int j = 0; j < ARRAY_LEN; j++) {
            matrix[i][j] = i * 100 + j;
        }
    }

    int sum = sum_matrix(matrix);
    printf("%d\n", sum);
}

10225
